In [ ]:
!pip install opencv-python

In [ ]:
!pip install pyttsx3

In [1]:
# Importing Libraries
import cv2
import numpy as np
import pyttsx3

In [2]:
# Initialize the text-to-speech engine
engine = pyttsx3.init()

In [3]:
# YOLO configuration
yolo_config = "yolov.cfg"  

In [4]:
yolo_weights = "yolov.weights"  

In [5]:
yolo_classes = "coco.names"  

In [6]:
# Load YOLO network
net = cv2.dnn.readNet(yolo_weights, yolo_config)

In [7]:
# Load YOLO classes
with open(yolo_classes, "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [8]:
# Initialize the video capture
cap = cv2.VideoCapture(0)  # Use 0 for default camera

In [ ]:
while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # YOLO object detection
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())
    
    # Process YOLO results
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Voice feedback
    feedback_text = ""
    for i in range(len(boxes)):
        if i in indexes:
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            feedback_text += f"{label}: {confidence:.2f}\n"

    # Display bounding boxes on the frame
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the frame
    cv2.imshow("Live Feed Object Detection using YOLO Algorithm with Audio Feedback Mechanism", frame)

    # Speak the feedback
    engine.say(feedback_text)
    engine.runAndWait()

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()